<a href="https://colab.research.google.com/github/EbsHirani/redditChatbot/blob/master/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
#os.chdir('drive/My Drive/')
#os.mkdir('chatbot')
os.chdir('chatbot')
os.listdir()

['RC_2015-08.bz2']

In [0]:
import sqlite3
import json
from datetime import datetime
timeframe = '2015-08'
connection = sqlite3.connect(f"{timeframe}.db")
c = connection.cursor()
c.execute("create table if not exists parent_reply(parent_id text primary key, comment_id text unique, parent text, comment text, subreddit text, unix int, score int )")



In [0]:
query_list= []
row_counter = 0
pairs = 0
def format_data(data):
  data = data.replace("\n",' newline ').replace('\r',' newline ').replace('"',"'")
def find_parent(parent_id):
  query = f"select comment from parent_reply where comment_id = '{parent_id}' limit 1"
  try:
    c.execute(query)
    result = c.fetchone()
    if result != None:
      return result[0]
    else:  
      return False
  except Exception as e:  
     return False 
def find_existing_parent(parent_id):
  query = f"select score from parent_reply where parent_id = '{parent_id}' limit 1"
  try:
    c.execute(query)
    result = c.fetchone()
    if result != None:
      return result[0]
    else:  
      return False
  except Exception as e:  
     return False 
def acceptable(data):
  if len(data.split(" ")) > 50 or len(data.split(" ")) < 1:
    return False
  elif  len(data) > 1000:
    return False
  elif data == '[deleted]' or data == '[removed]':
    return False
  else:
    return True
def add_query(query):
  global query_list
  query_list.append(query)
  if len(query_list) > 1000:
    c.execute("begin transaction")
    for query in query_list:
      try :
        c.execute(query)
      except Exception as e:
        print('error in ' + query)
    connection.commit()
    query_list = []



def insert_has_parent(comment_id, parent_id, parent_data, body, subreddit, created_utc, score):
  try:
    query = f"insert into table parent_reply values({parent_id}, {comment_id}, {parent_data}, {body}, {subreddit}, {int(created_utc)}, {score})"
    add_query(query)
  except Exception as e:
    print("parent insert error")
def insert_no_parent(comment_id,parent_id,body,subreddit, created_utc, score):
  try:
    query = f"insert into table parent_reply(parent_id, comment_id, comment, subreddit, unix, score) values({parent_id}, {comment_id}, {body}, {subreddit}, {int(created_utc)}, {score})"
    add_query(query)
  except Exception as e:
    print("no parent insert error")
def insert_replace(comment_id,parent_id,parent_data,body,subreddit, created_utc, score):
  try:
    query = f"update parent_reply set parent_id = {parent_id}, comment_id = {comemnt_id}, parent = {parent_data}, comment = {body}, subreddit = {subreddit}, unix = {int(created_utc)}, score = {score} where parent_id = {parent_id}"
    add_query(query)
  except Exception as e:
    print("insert replace error")


In [0]:
with open(f'RC_{timeframe}',buffering = 1000) as f:
  for row in f:
    row_counter += 1
    row = json.loads(row)
    parent_id = row['parent_id']
    body = format_data(row['body'])
    created_utc = row["created_utc"]
    score = row["score"]
    comment_id = row["name"]
    subreddit = row['subreddit']
    parent_data = find_parent(parent_id)
    if acceptable(body):
      if score >= 2:
        existing_score = find_existing_parent(parent_id)
        if existing_score:
          if score > existing_score:
            insert_replace(comment_id,parent_id,parent_data,body,subreddit, created_utc, score)
        else:
          if parent_data:
            insert_has_parent(comment_id,parent_id,parent_data,body,subreddit, created_utc, score)
            pairs += 1
          else:
            insert_no_parent(comment_id, parent_id, body,subreddit, created_utc, score)
    if row_counter %100000 == 0 :
      print(f"rows = {row_counter} pairs = {pairs}")


In [0]:
import pandas as pd
timeframes = ['2015-08']
for timeframe in timeframes:
  limit = 5000
  last_time = 0
  read_len = 5000
  count = 0
  test_stat = False 
  while limit == read_len:
    data = pd.read_sql(f"select * from parent_reply where unix > {last_time} and parent not null and score > 0 order by unix asc limit {limit}",connection)
    last_time = data.tail(1)['unix'].values[0]
    if not test_stat:
      with open("test.from", 'a', encoding = 'utf8') as f:
        for content in data["parent"].values:
          f.write(content + '\n')
      with open("test.to", 'a', encoding = 'utf8') as f:
        for content in data["comment"].values:
          f.write(content + '\n')
      test_stat = True
    else:
      with open("train.from", 'a', encoding = 'utf8') as f:
        for content in data["parent"].values:
          f.write(content + '\n')
        for content in data['comment'].values:
          f.write(content + '\n') 
    count += 1
    if count%10 == 0:
      print(count * limit, 'rows')

                

In [0]:
def clean_text(text):
    

    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    
    return text


In [0]:
from keras.preprocessing.text import Tokenizer

def tokenize(inp, out):
  tokenizer = Tokenizer(char_level = False)
  
  tokenizer.fit_on_texts(inp + out)
  return tokenizer.texts_to_sequences(inp), tokenizer.texts_to_sequences(out), tokenizer


In [0]:
def pad(x, length = None):
  if length is None:
    length  = max([len(i) for i in x])
    print(length)
  return pad_sequences(x, maxlen = length, padding = 'post')

In [0]:
def preprocess(x, y):
  x.apply(clean_text)
  y.apply(clean_text)
  preprocess_x, preprocess_y, xtk = tokenize(x)
  
  preprocess_x = pad(preprocess_x)
  preprocess_y = pad(preprocess_y)
  preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)
  return preprocess_x, preprocess_y, x_tk


In [0]:
pro_in, pro_out, eng_tk = preprocess(in_sentences, out_sentences)
max_in_len = pro_in.shape[1]
max_out_len = pro_out.shape[1]
eng_vocab_size = len(eng_tk.word_index)

In [0]:
def output_to_text(output, tokenizer):
  index_to_words = {id: word for word , id in tokenizer.word_index.items()}
  index_to_words[0] = "<PAD>"
  return " ".join([index_to_words[pr] for pr in np.argmax(output,1)])

In [0]:
def create_model(input_shape, output_length, eng_voc_size):
  lr = 1e-3
  model = Sequential{
      [
       Embedding(eng_voc_size, 128, input_length = input_shape[1]),
       Bidirectional(LSTM(128, return_sequences = False)),
       RepeatVector(output_length),
       Bidirectional(LSTM(128, return_sequences = True)),
       TimeDistributed(Dense(eng_vocab_size, activation = 'softmax'))
      ]
  }
  model.summary()
  model.compile(loss = sparse_categorical_crossentropy, optimizer = Adam(lr), metrics = ['accuracy'])
  return model

In [0]:
input_x = pad(pro_in, max(max_in_len, max_out_len))
output_y = pad(pro_out, max(max_out_len, max_in_len))
fin_model = create_model(input_x.shape, max(max_out_len, max_in_len), eng_voc_size)
history = fin_model.fit(input_x, output_y, batch_size = 256, epochs = 50, validation_split = 0.05)


In [0]:
def tagger(decoder_input_sentence):
  bos = "<BOS> "
  eos = " <EOS>"
  final_target = [bos + text + eos for text in decoder_input_sentence] 
  return final_target

decoder_inputs = tagger(decoder_input_text)

In [0]:
# from keras.preprocessing.text import Tokenizer

# def vocab_creater(text_lists, VOCAB_SIZE):

#   tokenizer = Tokenizer(num_words=VOCAB_SIZE)
#   tokenizer.fit_on_texts(text_lists)
#   dictionary = tokenizer.word_index
  
#   word2idx = {}
#   idx2word = {}
#   for k, v in dictionary.items():
#       if v < VOCAB_SIZE:
#           word2idx[k] = v
#           index2word[v] = k
#       if v >= VOCAB_SIZE-1:
#           continue
          
#   return word2idx, idx2word

# word2idx, idx2word = vocab_creater(text_lists=encoder_input_text+decoder_input_text, VOCAB_SIZE=14999)


In [0]:
# from keras.preprocessing.text import Tokenizer
# VOCAB_SIZE = 14999

# def text2seq(encoder_text, decoder_text, VOCAB_SIZE):

#   tokenizer = Tokenizer(num_words=VOCAB_SIZE)
#   encoder_sequences = tokenizer.texts_to_sequences(encoder_text)
#   decoder_sequences = tokenizer.texts_to_sequences(decoder_text)
  
#   return encoder_sequences, decoder_sequences

# encoder_sequences, decoder_sequences = text2seq(encoder_text, decoder_text, VOCAB_SIZE) 


In [0]:
# from keras.preprocessing.sequence import pad_sequences

# def padding(encoder_sequences, decoder_sequences, MAX_LEN):
  
#   encoder_input_data = pad_sequences(encoder_sequences, maxlen=MAX_LEN, dtype='int32', padding='post', truncating='post')
#   decoder_input_data = pad_sequences(decoder_sequences, maxlen=MAX_LEN, dtype='int32', padding='post', truncating='post')
  
#   return encoder_input_data, decoder_input_data

# encoder_input_data, decoder_input_data = padding(encoder_sequences, decoder_sequences, MAX_LEN):


In [0]:
# def seq2seq_model_builder(HIDDEN_DIM=300):
    
#     encoder_inputs = Input(shape=(MAX_LEN, ), dtype='int32',)
#     encoder_embedding = embed_layer(encoder_inputs)
#     encoder_LSTM = LSTM(HIDDEN_DIM, return_state=True)
#     encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)
    
#     decoder_inputs = Input(shape=(MAX_LEN, ), dtype='int32',)
#     decoder_embedding = embed_layer(decoder_inputs)
#     decoder_LSTM = LSTM(HIDDEN_DIM, return_state=True, return_sequences=True)
#     decoder_outputs, _, _ = decoder_LSTM(decoder_embedding, initial_state=[state_h, state_c])
    
#     # dense_layer = Dense(VOCAB_SIZE, activation='softmax')
#     outputs = TimeDistributed(Dense(VOCAB_SIZE, activation='softmax'))(decoder_outputs)
#     model = Model([encoder_inputs, decoder_inputs], outputs)
    
#     return model